# Data Analysis
- Range of Motion</br>
  Using the MoCap data, determine how far the phone / head is moved for each gesture (broken down by head and phone, linear and angular, and participant)
- Time Taken</br>
  How long it takes for the participant to perform a specific gesture (broken down by gesture and direction)</br>
  How to determine when gesture started vs finished (2 metrics, from first line to last, from motion started to stopped (threshold for motion?))
- Frame Containing Face</br>
  Use Open CV, how determine rotated faces, manually mark frames between frames with faces marked as not having any?</br>
  TensorFlow model for face detection?
- Average Sample Rate for each data source (image, imu, ignore MoCap as fixed rate), also split by attempt


In [1]:
from random import shuffle
import csv
import math
import cv2 as cv
import numpy as np
import torch
from pathlib import Path
import itertools as iter
import importlib
from statistics import mean
# md = importlib.import_module(".mark_detector","assets.head-pose-estimation-master")

# Do below as TFLite model? https://stackoverflow.com/questions/53182177/how-do-you-convert-a-onnx-to-tflite#58576060
# https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/opencv_face_detector.pbtxt
detector = cv.FaceDetectorYN.create(
        'assets/face_detection_yunet_2022mar-act_int8-wt_int8-quantized.onnx',
        "",
        (180, 320),
        0.9,
        0.3,
        5000
    )
detector.setInputSize((180, 320))

# mark_detector = md.MarkDetector(saved_model='assets/head-pose-estimation-master/assets/pose_model',dnn_proto_text='assets/head-pose-estimation-master/assets/deploy.prototxt', dnn_model='assets/head-pose-estimation-master/assets/res10_300x300_ssd_iter_140000.caffemodel')

DATA_PATH = Path('/mnt/e/Uni/CM50175/Study/data')
OUTPUT_PATH = DATA_PATH / 'training_data'
NN_PATH = OUTPUT_PATH / 'NN'
NN_MOTION_PATH = OUTPUT_PATH / 'NN' / 'POSE'
NN_POSE_PATH = OUTPUT_PATH / 'NN' / 'MOTION'
HMM_PATH = OUTPUT_PATH / 'HMM'
ANALYSIS_OUTPUT_FILE_PATH = DATA_PATH / 'analysis.csv'

FPS = [5,10,15,20]

NN_MOTION_PATH.mkdir(parents=True, exist_ok=True)
NN_POSE_PATH.mkdir(parents=True, exist_ok=True)

ANALYSIS_CSV_HEADERS = [
    'GESTURE', 'DIRECTION', 'PARTICIPANT', 'TAKE', 'AVERAGE_IMG_SAMPLE_RATE', 'AVERAGE_IMU_SAMPLE_RATE', 'IMAGE_COUNT', 'PERCENTAGE_FRAMES_HEAD_DETECTED', 'TIME_ELAPSED',
    'HEAD_X', 'HEAD_Y', 'HEAD_Z', 'HEAD_PITCH', 'HEAD_ROLL', 'HEAD_YAW', 'HEAD_X_ABS', 'HEAD_Y_ABS', 'HEAD_Z_ABS', 'HEAD_PITCH_ABS', 'HEAD_ROLL_ABS', 'HEAD_YAW_ABS', 
    'PHONE_X', 'PHONE_Y', 'PHONE_Z', 'PHONE_PITCH', 'PHONE_ROLL', 'PHONE_YAW', 'PHONE_X_ABS', 'PHONE_Y_ABS', 'PHONE_Z_ABS', 'PHONE_PITCH_ABS', 'PHONE_ROLL_ABS', 'PHONE_YAW_ABS',
    'LINEAR_X', 'LINEAR_Y', 'LINEAR_Z', 'ROTATION_PITCH', 'ROTATION_ROLL', 'ROTATION_YAW',
    'LINEAR_X_ABS', 'LINEAR_Y_ABS', 'LINEAR_Z_ABS', 'ROTATION_PITCH_ABS', 'ROTATION_ROLL_ABS', 'ROTATION_YAW_ABS'
]

INPUT_CSV_FIELD_NAMES = [
    'MS_FROM_START', 'MS_DELTA', 'RELATIVE_IMAGE_PATH', 'LINEAR_X', 'LINEAR_Y', 'LINEAR_Z', 'ROTATION_X', 'ROTATION_Y', 'ROTATION_Z', 'ROTATION_SCALAR', 
    'LINEAR_X_DELTA', 'LINEAR_Y_DELTA', 'LINEAR_Z_DELTA', 'ROTATION_X_DELTA', 'ROTATION_Y_DELTA', 'ROTATION_Z_DELTA', 'ROTATION_SCALAR_DELTA',
    'HEAD_X', 'HEAD_Y', 'HEAD_Z', 'HEAD_PITCH', 'HEAD_ROLL', 'HEAD_YAW', 'PHONE_X', 'PHONE_Y', 'PHONE_Z', 'PHONE_PITCH', 'PHONE_ROLL', 'PHONE_YAW'
]

INPUT_CSV_MS_FIELD_NAMES = ['IMAGE_DELTA_MS', 'IMU_DELTA_MS']

# CSV_CLASSIFIER_FIELD_NAMES = []
INPUT_CSV_REGRESSION_FIELD_NAMES = [
    'HEAD_X', 'HEAD_Y', 'HEAD_Z', 'HEAD_PITCH', 'HEAD_ROLL', 'HEAD_YAW', 'PHONE_X', 'PHONE_Y', 'PHONE_Z', 'PHONE_PITCH', 'PHONE_ROLL', 'PHONE_YAW'
]

OUTPUT_CSV_FACE_LM_FIELD_NAMES = [
    *["LM_{}".format(idx) for idx in range(0, 68)], # Landmark Points
    'HEAD_PRESENT', 'HEAD_BB_X', 'HEAD_BB_WIDTH', 'HEAD_BB_Y', 'HEAD_BB_HEIGHT', # BB = Bounding Box
    *["BB_LM_{}".format(idx) for idx in range(0, 68)] # Landmark Points
]

YUNET_FIELDS = [
    'BB_X','BB_WIDTH','BB_Y','BB_HEIGHT','BB_EYE_R_X','BB_EYE_R_Y','BB_EYE_L_X','BB_EYE_L_Y',
    'BB_NOSE_X','BB_NOSE_Y','BB_MOUTH_R_X','BB_MOUTH_R_Y','BB_MOUTH_L_X','BB_MOUTH_L_Y'
]

OUTPUT_CSV_FIELD_NAMES = [
    'IMG_PATH', 'FACE_DETECTED', *YUNET_FIELDS, 
    'LINEAR_X', 'LINEAR_Y', 'LINEAR_X', 'ROTATION_X', 'ROTATION_Y', 'ROTATION_Z',
    'PHONE_X', 'PHONE_Y', 'PHONE_Z', 'PHONE_ROLL', 'PHONE_PITCH', 'PHONE_YAW', 
    'HEAD_X', 'HEAD_Y', 'HEAD_Z', 'HEAD_ROLL', 'HEAD_PITCH', 'HEAD_YAW'
]

AGG_OUTPUT_CSV_FIELD_NAMES = ['DELTA', *OUTPUT_CSV_FIELD_NAMES]

MOTION_OUTPUT_CSV_FIELD_NAMES = [
    'IMG_PATH_1', 'FACE_DETECTED_1', *[f"{field}_1" for field in YUNET_FIELDS], 
    'IMG_PATH_2', 'FACE_DETECTED_2', *[f"{field}_2" for field in YUNET_FIELDS], 
    'LINEAR_X', 'LINEAR_Y', 'LINEAR_X', 'ROTATION_X', 'ROTATION_Y', 'ROTATION_Z',
    'PHONE_X', 'PHONE_Y', 'PHONE_Z', 'PHONE_ROLL', 'PHONE_PITCH', 'PHONE_YAW', 
    'HEAD_X', 'HEAD_Y', 'HEAD_Z', 'HEAD_ROLL', 'HEAD_PITCH', 'HEAD_YAW'
]

AGG_MOTION_OUTPUT_CSV_FIELD_NAMES = ['DELTA', *MOTION_OUTPUT_CSV_FIELD_NAMES]

ROTATION_KEYS = {
    'HEAD_PITCH', 'HEAD_ROLL', 'HEAD_YAW', 
    'PHONE_PITCH', 'PHONE_ROLL', 'PHONE_YAW'
}

def file_path_generator(root_dir):
    for participant in range(0,8):
        participant_path = DATA_PATH / str(participant)
        if participant_path.is_dir():
            synced_data_path = participant_path / 'synced_data'
            for take in range(0,3):
                take_path = participant_path / f'attempt_{take}'
                if take_path.is_dir():
                    for gesture in take_path.iterdir():
                        for direction in gesture.iterdir():
                            if len(list((direction / 'images').iterdir())) > 0:
                                csv_file = synced_data_path / f'{take_path.name}_{gesture.name}_{direction.name}.csv'
                                if Path(csv_file).is_file():
                                    yield (participant, take, gesture.name, direction.name, csv_file)

# https://automaticaddison.com/how-to-convert-a-quaternion-into-euler-angles-in-python/
def euler_from_quaternion(x, y, z, w):
    """
    Convert a quaternion into euler angles (roll, pitch, yaw)
    roll is rotation around x in radians (counterclockwise)
    pitch is rotation around y in radians (counterclockwise)
    yaw is rotation around z in radians (counterclockwise)
    """
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    roll_x = math.atan2(t0, t1)
    
    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    pitch_y = math.asin(t2)
    
    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    yaw_z = math.atan2(t3, t4)
    
    return np.array([roll_x, pitch_y, yaw_z]) # in radians

# https://automaticaddison.com/how-to-convert-euler-angles-to-quaternions-using-python/
def get_quaternion_from_euler(roll, pitch, yaw):
    """
    Convert an Euler angle to a quaternion.
     
    Input
      :param roll: The roll (rotation around x-axis) angle in radians.
      :param pitch: The pitch (rotation around y-axis) angle in radians.
      :param yaw: The yaw (rotation around z-axis) angle in radians.
   
    Output
      :return qx, qy, qz, qw: The orientation in quaternion [x,y,z,w] format
    """
    qx = np.sin(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) - np.cos(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    qy = np.cos(roll/2) * np.sin(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.cos(pitch/2) * np.sin(yaw/2)
    qz = np.cos(roll/2) * np.cos(pitch/2) * np.sin(yaw/2) - np.sin(roll/2) * np.sin(pitch/2) * np.cos(yaw/2)
    qw = np.cos(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
   
    return np.quaternian(qw, qx, qy, qz)

# # from https://docs.opencv.org/4.x/d0/dd4/tutorial_dnn_face.html
# def visualize(input, faces, thickness=2):
#     if faces[1] is not None:
#         for idx, face in enumerate(faces[1]):
#             print('Face {}, top-left coordinates: ({:.0f}, {:.0f}), box width: {:.0f}, box height {:.0f}, score: {:.2f}'.format(idx, face[0], face[1], face[2], face[3], face[-1]))
#             coords = face[:-1].astype(np.int32)
#             cv.rectangle(input, (coords[0], coords[1]), (coords[0]+coords[2], coords[1]+coords[3]), (0, 255, 0), thickness)
#             cv.circle(input, (coords[4], coords[5]), 2, (255, 0, 0), thickness)
#             cv.circle(input, (coords[6], coords[7]), 2, (0, 0, 255), thickness)
#             cv.circle(input, (coords[8], coords[9]), 2, (0, 255, 0), thickness)
#             cv.circle(input, (coords[10], coords[11]), 2, (255, 0, 255), thickness)
#             cv.circle(input, (coords[12], coords[13]), 2, (0, 255, 255), thickness)
    

/home/whiffingj/bin/anaconda3/envs/diss/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import importlib
from statistics import mean
md = importlib.import_module(".mark_detector","assets.head-pose-estimation-master")

# Do below as TFLite model? https://stackoverflow.com/questions/53182177/how-do-you-convert-a-onnx-to-tflite#58576060
# https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/opencv_face_detector.pbtxt
detector = cv.FaceDetectorYN.create(
        'assets/face_detection_yunet_2022mar-act_int8-wt_int8-quantized.onnx',
        "",
        (180, 320),
        0.9,
        0.3,
        5000
    )
detector.setInputSize((180, 320))

mark_detector = md.MarkDetector(saved_model='assets/head-pose-estimation-master/assets/pose_model',dnn_proto_text='assets/head-pose-estimation-master/assets/deploy.prototxt', dnn_model='assets/head-pose-estimation-master/assets/res10_300x300_ssd_iter_140000.caffemodel')

# https://github.com/yinguobing/head-pose-estimation
def get_landmarks(img):
    # Run the detection.
    marks = mark_detector.detect_marks(img) # TODO: double check output

    # Try pose estimation with 68 points.
    # pose = pose_estimator.solve_pose_by_68_points(marks)
    return marks

def process_img(img_path):
    img = cv.imread(str(img_path))
    img = cv.resize(img, (320, 180))
    img = np.rot90(img)
    img = np.ascontiguousarray(img)
    bounding_box = detector.detect(img)[1]
    bounding_box_landmarks = []
    landmarks = None

    # cascade, non-cascade
    return (None if bounding_box is None else [*bounding_box, *bounding_box_landmarks], landmarks)

with open(ANALYSIS_OUTPUT_FILE_PATH, 'w') as analysis_file:
    analysis_writer = csv.writer(analysis_file)
    analysis_writer.writerow(ANALYSIS_CSV_HEADERS)
    for participant, take, gesture, direction, file in file_path_generator(DATA_PATH):
        print(f"Processing: {participant}-{take}-{gesture}-{direction}")
        with open(file, 'r') as csv_file:
            reader = csv.DictReader(csv_file)
            # read sheet into memory
            # for each output type, run through the data      
            img_sample_rates = []
            imu_sample_rates = []
            head_detected = 0
            img_count = 0
            final_ms = 0

            # TODO: Some logic to correctly determine yaw based on whether diff is greater than pi between 2 frames, 
            #   need to take diff between frames and sum them to get range
            #   if current is > pi from prior, take 2*pi - current val, and vice-versa
            # TODO: Convert Quaternion into Euler Angles (can eyeball whether rotating same amount)
            # min, max
            linear_extremes = {
                'HEAD_X' : (np.Inf,-np.Inf), 'HEAD_Y' : (np.Inf,-np.Inf), 'HEAD_Z' : (np.Inf,-np.Inf), 
                'PHONE_X' : (np.Inf,-np.Inf), 'PHONE_Y' : (np.Inf,-np.Inf), 'PHONE_Z' : (np.Inf,-np.Inf), 
                'LINEAR_X' : (np.Inf,-np.Inf), 'LINEAR_Y' : (np.Inf,-np.Inf), 'LINEAR_Z' : (np.Inf,-np.Inf)
            }

            # previous, delta, abs
            linear_deltas = {
                'HEAD_X' : None, 'HEAD_Y' : None, 'HEAD_Z' : None, 
                'PHONE_X' : None, 'PHONE_Y' : None, 'PHONE_Z' : None, 
                'LINEAR_X' : None, 'LINEAR_Y' : None, 'LINEAR_Z' : None
            }

            # previous, delta, abs
            rotation_deltas = {
                'HEAD_PITCH' : None, 'HEAD_ROLL' : None, 'HEAD_YAW' : None, 
                'PHONE_PITCH' : None, 'PHONE_ROLL' : None, 'PHONE_YAW' : None,
                'ROTATION_PITCH' : None, 'ROTATION_ROLL' : None, 'ROTATION_YAW' : None
            }

            ranges = {
                'HEAD_X' : 0, 'HEAD_Y' : 0, 'HEAD_Z' : 0, 'HEAD_PITCH' : 0, 'HEAD_ROLL' : 0, 'HEAD_YAW' : 0, 
                'HEAD_X_ABS' : 0, 'HEAD_Y_ABS' : 0, 'HEAD_Z_ABS' : 0, 'HEAD_PITCH_ABS' : 0, 'HEAD_ROLL_ABS' : 0, 'HEAD_YAW_ABS' : 0, 
                'PHONE_X' : 0, 'PHONE_Y' : 0, 'PHONE_Z' : 0, 'PHONE_PITCH' : 0, 'PHONE_ROLL' : 0, 'PHONE_YAW' : 0,
                'PHONE_X_ABS' : 0, 'PHONE_Y_ABS' : 0, 'PHONE_Z_ABS' : 0, 'PHONE_PITCH_ABS' : 0, 'PHONE_ROLL_ABS' : 0, 'PHONE_YAW_ABS' : 0,
                'LINEAR_X' : 0, 'LINEAR_Y' : 0, 'LINEAR_Z' : 0, 'ROTATION_PITCH' : 0, 'ROTATION_ROLL' : 0, 'ROTATION_YAW' : 0,
                'LINEAR_X_ABS' : 0, 'LINEAR_Y_ABS' : 0, 'LINEAR_Z_ABS' : 0, 'ROTATION_PITCH_ABS' : 0, 'ROTATION_ROLL_ABS' : 0, 'ROTATION_YAW_ABS' : 0
            }

            previous_img_path = ""
            for row in reader: # presuming already read headers
                img_path = file.parents[0] / f"{row['RELATIVE_IMAGE_PATH'][2:]}.png"
                if img_path != previous_img_path:
                    previous_img_path = img_path
                    img_count += 1
                    img_sample_rates.append(float(row[INPUT_CSV_MS_FIELD_NAMES[0]]))
                    cascade, non_cascade = process_img(img_path)
                    if cascade is not None:
                        head_detected += 1
                imu_sr = float(row[INPUT_CSV_MS_FIELD_NAMES[1]])
                if imu_sr > 0: imu_sample_rates.append(imu_sr)
                
                for key, value in linear_extremes.items():
                    new_value = float(row[key])
                    if new_value < value[0]:
                        ranges[key] = (new_value, value[1])
                    elif new_value > value[1]:
                        ranges[key] = (value[0], new_value)
                
                row['ROTATION_PITCH'], row['ROTATION_ROLL'], row['ROTATION_YAW'] = euler_from_quaternion(float(row['ROTATION_X']), float(row['ROTATION_Y']), float(row['ROTATION_Z']), float(row['ROTATION_SCALAR']))
                
                for key, value in rotation_deltas.items():
                    current_rotation = float(row[key])
                    if value is None:
                        rotation_deltas[key] = (current_rotation, 0, 0)
                    else:
                        delta = current_rotation - value[0]
                        if delta > 2 * math.pi: # rotated > 180degrees (not feasible in the timestamps we have)
                            delta = (2 * math.pi) - delta # subtract 360 Degrees, as more likely rotation went below 0/-pi
                        elif delta < -2 * math.pi:
                            delta = (-2 * math.pi) - delta # add 360 Degrees, as more likely rotation went below 0/-pi
                        rotation_deltas[key] = (current_rotation, value[1] + delta, value[2] + abs(delta))
                
                for key, value in linear_deltas.items():
                    current_pos = float(row[key])
                    if value is None:
                        linear_deltas[key] = (current_pos, 0, 0)
                    else:
                        delta = current_pos - value[0]
                        linear_deltas[key] = (current_pos, value[1] + delta, value[2] + abs(delta))

                final_ms = row['MS_FROM_START']
                    
            img_average = mean(img_sample_rates)
            imu_average = mean(imu_sample_rates)
            for key, (max, min) in linear_extremes.items():
                ranges[key] = max - min
            for data in [rotation_deltas, linear_deltas]:
                for key, deltas in data.items():
                    ranges[key] = deltas[1]
                    ranges[f'{key}_ABS'] = deltas[2]
                    
            analysis_writer.writerow([
                gesture, direction, participant, take, img_average, imu_average, img_count, head_detected/img_count, final_ms,
                *ranges.values()
            ])


In [ ]:
# test landmark extraction
img_path = DATA_PATH / '4' / 'synced_data' / 'attempt_1' / 'ROTATE_HEAD_ROLL' / 'CLOCKWISE' / 'images'
def preview_img(path):
    def load_img():
        img = cv.imread(str(path))
        img = cv.resize(img, (320, 180))
        img = np.rot90(img)
        return np.ascontiguousarray(img)
    
    bb_img = load_img()
    bbox = detector.detect(bb_img)[1]
    if bbox is not None:
        x1, y1, x2, y2 = bbox[0][0:4].astype(np.int32)
        print(f"bounding box found: {bbox} - x1 {x1}, y1 {y1}, x2 {x2}, y2 {y2}")
        face_img = bb_img[y1: y1+y2, x1: x1+x2]
        cv.imshow("Face Image:", face_img)
        bb_lm = mark_detector.detect_marks(face_img)
        bb_lm[:, 0] *= x2
        bb_lm[:, 1] *= y2
        mark_detector.draw_marks(face_img, bb_lm)
        for x, y in bbox[0][4:-1].astype(np.int32).reshape((5,2)):
            cv.circle(bb_img, (x,y), 1, (0, 255, 0))
        cv.imshow("Face Image:", face_img)
        cv.rectangle(bb_img, (x1, y1), (x1+x2, y1+y2), (0, 255, 0), 2)
        cv.imshow("Face Found:", bb_img)
    else:
        cv.imshow("Face Not Found:", bb_img)

    lm_img = load_img()
    lm = mark_detector.detect_marks(lm_img)
    lm[:, 0] *= 180
    lm[:, 1] *= 320
    print(lm)
    mark_detector.draw_marks(lm_img, lm)
    cv.imshow("Landmarks without cascade", lm_img)

for img_name in ['2022-08-23T10-02-32-315.png', '2022-08-23T10-02-32-693.png', '2022-08-23T10-02-32-726.png']:
    preview_img(img_path / img_name)
    cv.waitKey(0)


In [57]:
cv.destroyAllWindows()

In [9]:
import importlib
import math
from statistics import mean
import quaternion
# md = importlib.import_module(".mark_detector","assets.head-pose-estimation-master")
# pe = importlib.import_module(".pose_estimator","assets.head-pose-estimation-master")

# Data we want:
# - One file with all deltas with:
#   Input:
#   - sensor data (all but bias/offset) (for both timestamps), 
#   - image path (for both timestamps),
#   - image landmarks (for both timestamps),
#   - delta between frames
#   Output:
#   - one-hot encoded movement ([neg, n/a, pos]), 
#     - encoded by quantising mocap data (multiply position by 100 and round to nearest number, multiply rotation by 10 and round to nearest number)
#       If diff between quantised numbers is positive: [0,0,1], if negative: [1,0,0], if 0: [0,1,0] (how to write to file? 0|1|0, then split on '|')
#   Generate for fps limited as well (e.g. average the deltas in acceleration and rotation till next frame)
# - files for each gesture performed:
#   Input:
#   - sensor data (all but bias/offset), 
#   - image path,
#   - image landmarks,
#   - delta between frames
#   Output
#   - file name

# Do below as TFLite model? https://stackoverflow.com/questions/53182177/how-do-you-convert-a-onnx-to-tflite#58576060
# https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/opencv_face_detector.pbtxt
detector = cv.FaceDetectorYN.create(
        'face_detection_yunet_2022mar-act_int8-wt_int8-quantized.onnx',
        "",
        (180, 320),
        0.9,
        0.3,
        5000
    )
detector.setInputSize((180, 320))

# mark_detector = md.MarkDetector(saved_model='assets/head-pose-estimation-master/assets/pose_model',dnn_proto_text='assets/head-pose-estimation-master/assets/deploy.prototxt', dnn_model='assets/head-pose-estimation-master/assets/res10_300x300_ssd_iter_140000.caffemodel')

# https://github.com/yinguobing/head-pose-estimation
# def get_landmarks(img):
#     # Run the detection.
#     marks = mark_detector.detect_marks(img)

#     # Try pose estimation with 68 points.
#     # pose = pose_estimator.solve_pose_by_68_points(marks)
#     return marks

def process_img(img_path):
    img = cv.imread(str(img_path))
    img = cv.resize(img, (320, 180))
    img = np.rot90(img)
    img = np.ascontiguousarray(img)
    bounding_box = detector.detect(img)[1]
    return bounding_box

# require quantised pose to work
def encode_motion(new, previous):
    encoded = []
    for a, b in np.column_stack(new, previous):
        diff = a - b
        if diff > 0:
            encoded.append([0,0,1])
        elif diff < 0:
            encoded.append([1,0,0])
        else:
            encoded.append([0,1,0])

def get_rotation_delta(a, b):
    if np.dot(a, b) < 0:
        return a * np.inv(b * -1)
    else:
        return a * np.inv(b)

#     *["LM_{}".format(idx) for idx in range(0, 68)], # Landmark Points
#     'HEAD_PRESENT', 'HEAD_BB_X', 'HEAD_BB_WIDTH', 'HEAD_BB_Y', 'HEAD_BB_HEIGHT', # BB = Bounding Box
#     *["BB_LM_{}".format(idx) for idx in range(0, 68)] # Landmark Points
# INPUT_CSV_REGRESSION_FIELD_NAMES = [
#     'HEAD_X', 'HEAD_Y', 'HEAD_Z', 'HEAD_PITCH', 'HEAD_ROLL', 'HEAD_YAW', 'PHONE_X', 'PHONE_Y', 'PHONE_Z', 'PHONE_PITCH', 'PHONE_ROLL', 'PHONE_YAW'
# ]

# want to quantise, avoid working with numbers smaller than 0
def quantise_position(vector):
    return [int(round(value * 100)) for value in vector]
def quantise_acceleration(vector):
    return [int(round(value*5)) for value in vector]
# up to 20 values (0-19)
def quantise_pose(vector):
    return [int(round(value * 3)) for value in vector]

# just extract data and pre-process?
def process_row(row, aggregated_data, path, previous_data = None):
    # data = {key: row[key] for key in [*INPUT_CSV_FIELD_NAMES, *INPUT_CSV_REGRESSION_FIELD_NAMES]} # TODO: update keys
    previous_processed_row, previous_training_data = previous_data

    # TODO: aggregated_data is for tracking a list of the prior frames of data between images/frame-rates.
    # should include key for each aggregation (RAW, FPS5...FPS20), 
    # the values should be a tuple of: the timestamp of the last image, and a list of the prior data from which to calculate the delta

    # check if new data exists for each type
    ms_from_start = row['MS_FROM_START'] 
    imu_data_exists = row['IMU_DELTA_MS'] == 0
    img_data_exists = row['IMAGE_DELTA_MS'] == 0
    mocap_data_exists = row['MOCAP_DELTA_MS'] == 0

    # processes the deltas
    # 'ROTATION_X', 'ROTATION_Y', 'ROTATION_Z', 'ROTATION_SCALAR'
    # TODO: if quaternion delta is under some amount, keep old value in 'previous data', can then update and track movement once delta is exceeded, say 10 degrees, convert the quaternion delta back to euler and check the roll-pitch-yaw changes
    phone_imu_pose = np.quaternion(row['ROTATION_SCALAR'],row['ROTATION_X'],row['ROTATION_Y'],row['ROTATION_Z'])
    phone_mocap_pose = get_quaternion_from_euler(row['PHONE_ROLL'], row['PHONE_PITCH'], row['PHONE_YAW'])
    head_mocap_pose = get_quaternion_from_euler(row['HEAD_ROLL'], row['HEAD_PITCH'], row['HEAD_YAW'])

    # 'LINEAR_X', 'LINEAR_Y', 'LINEAR_Z'
    phone_imu_acc = quantise_acceleration(np.array([row['LINEAR_X'], row['LINEAR_Y'], row['LINEAR_Z']]))
    phone_mocap_loc = quantise_position(np.array([row['PHONE_X'], row['PHONE_Y'], row['PHONE_Z']]))
    head_mocap_loc = quantise_position(np.array([row['HEAD_X'], row['HEAD_Y'], row['HEAD_Z']]))

    # process image
    img_file = f"{row['RELATIVE_IMAGE_PATH'][2:]}.png"
    img_path = path.parents[0] / img_file
    img_data = {}
    if img_data_exists:
        img_sample_rates.append(float(row[INPUT_CSV_MS_FIELD_NAMES[0]]))
        bounding_box = process_img(img_path)
        if bounding_box is not None:
            img_data['HEAD_PRESENT'] = True
            img_data['BB_X'] = bounding_box[0]
            img_data['BB_WIDTH'] = bounding_box[1]
            img_data['BB_Y'] = bounding_box[2]
            img_data['BB_HEIGHT'] = bounding_box[3]
            img_data['BB_EYE_R_X'] = bounding_box[4]
            img_data['BB_EYE_R_Y'] = bounding_box[5]
            img_data['BB_EYE_L_X'] = bounding_box[6]
            img_data['BB_EYE_L_Y'] = bounding_box[7]
            img_data['BB_NOSE_X'] = bounding_box[8]
            img_data['BB_NOSE_Y'] = bounding_box[9]
            img_data['BB_MOUTH_R_X'] = bounding_box[10]
            img_data['BB_MOUTH_R_Y'] = bounding_box[11]
            img_data['BB_MOUTH_L_X'] = bounding_box[12]
            img_data['BB_MOUTH_L_Y'] = bounding_box[13]
        for key, data in aggregated_data.items():
            none_elapsed = True
            for old_img_timestamp, aggregate in data:
                if ms_from_start - old_img_timestamp > 1000 / key: # new data found for current aggregate
                    none_elapsed = False
            if none_elapsed:
                data.append() # append data from current frame

    processed_row = {
        'IMAGE_PATH': img_file,
    }

    if previous_data:
        # clone previous data, so we keep old data if no new data present
        training_data = {key: value for key, value in previous_training_data.items()}

        # quantize the angles
        if imu_data_exists:
            training_data['PHONE_IMU_POSE_DELTA'] = get_rotation_delta(previous_processed_row['PHONE_IMU_POSE'], phone_imu_pose)
            # need to average 
            training_data['PHONE_IMU_ACC_DELTA'] = previous_processed_row['PHONE_IMU_ACC'] - phone_imu_acc
                    
        if mocap_data_exists:
            training_data['PHONE_MOCAP_POSE_DELTA'] = get_rotation_delta(previous_processed_row['PHONE_MOCAP_POSE'], phone_mocap_pose)
            training_data['HEAD_MOCAP_POSE_DELTA'] = get_rotation_delta(previous_processed_row['HEAD_MOCAP_POSE'], head_mocap_pose)
            training_data['PHONE_MOCAP_LOC_DELTA'] = previous_processed_row['PHONE_MOCAP_LOC'] - phone_mocap_loc
            training_data['HEAD_MOCAP_LOC_DELTA'] = previous_processed_row['HEAD_MOCAP_LOC'] - head_mocap_loc
    else:
        # set training_data to current data
        training_data = {
            'PHONE_IMU_POSE_DELTA': phone_imu_pose,
            'PHONE_IMU_ACC_DELTA': phone_imu_acc,
            'PHONE_MOCAP_POSE_DELTA': phone_mocap_pose,
            'HEAD_MOCAP_POSE_DELTA': head_mocap_pose,
            'PHONE_MOCAP_LOC_DELTA': phone_mocap_loc,
            'HEAD_MOCAP_LOC_DELTA': head_mocap_loc,
            **processed_row,
        }

    
    
    return training_data, 

total_training_output_path = NN_PATH / 'Training.csv'
with open(total_training_output_path, 'w') as total_training_file:
    training_output_csv = csv.DictWriter(file, OUTPUT_CSV_FIELD_NAMES)
    for participant, take, gesture, direction, file in file_path_generator(DATA_PATH):
        print(f"Processing: {participant}-{take}-{gesture}-{direction}")
        with open(file, 'r') as csv_file:
            reader = csv.DictReader(csv_file)
            # read sheet into memory
            # for each output type, run through the data

            # files:
            #  - Collect each row, just need to calc diff from 'first frame'
            #  - mimic 5, 10, 15, 20 fps (average IMU & Mocap Acceleration data till next image after 1/Hz ms elapsed)
            # include rolling average of direction (1, 0, -1), use to 'split gesture' (can't do for circle gestures)
            # include threshold for 'movement' detected, can use to isolate when head and phone still
            previous_img_path = ""
            root_path = NN_PATH
            files = { 
                'RAW' : open(root_path / 'aggregated' / f'RAW_{file.name}', 'w')
                *{fps_file : open(root_path / 'aggregated' / f'{fps_file}_{file.name}', 'w') for fps_file in [f"FPS{fps}" for fps in FPS]}
            }
            writers = {key: csv.DictWriter(file, OUTPUT_CSV_REGRESSION_FIELD_NAMES if REGRESSION else OUTPUT_CSV_FIELD_NAMES) for key, file in files.items()}
            for writer in writers:
                writer.writeheader()
            ms_deltas = [(fps, 1000/fps) for fps in FPS]
            aggregated_data = {fps: {} for fps in FPS}
            try:
                for row in reader: # presuming already read headers
                    data = {key: row[key] for key in INPUT_CSV_FIELD_NAMES}
                    if REGRESSION: 
                        regression_data = {key: row[key] for key in INPUT_CSV_REGRESSION_FIELD_NAMES}
                    img_path = file.parents[0] / f"{row['RELATIVE_IMAGE_PATH'][2:]}.png"
                    if img_path != previous_img_path:
                        previous_img_path = img_path
                        img_count += 1
                        img_sample_rates.append(float(row[INPUT_CSV_MS_FIELD_NAMES[0]]))
                        cascade, non_cascade = process_img(img_path)
                        
                    for fps, agg_data in aggregated_data:
                        pass

                    # extract data

                    
            except Exception as e:
                print(f"Uh-oh: {e}", e.with_traceback)
            finally:
                for file in files.values():
                    file.close()
        

In [2]:
import importlib
import math
from statistics import mean
# import quaternion

# https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/opencv_face_detector.pbtxt
detector = cv.FaceDetectorYN.create(
        'assets/face_detection_yunet_2022mar-act_int8-wt_int8-quantized.onnx',
        "",
        (180, 320),
        0.9,
        0.3,
        5000
    )
detector.setInputSize((180, 320))

# require quantised pose to work
def encode_motion(new, previous):
    encoded = '0|1|0'
    diff = new - previous
    if diff > 0:
        encoded = '0|0|1'
    elif diff < 0:
        encoded = '1|0|0'
    return encoded

# def get_rotation_delta(a, b):
#     if np.dot(a, b) < 0:
#         return a * np.inv(b * -1)
#     else:
#         return a * np.inv(b)

# quantised angles in range of -9 - 9
def get_qauntised_rotation_delta(new, previous):
    diff = new - previous
    if diff > 9: # add anti-clockwise rotation
        diff = 18-diff+1
    elif diff < -9:
        diff = -18-diff-1
    return diff

# want to quantise, avoid working with numbers smaller than 0
def quantise_position(vector):
    return [int(round(value * 100)) for value in vector]

def quantise_acceleration(vector):
    return [int(round(value*5)) for value in vector]

# up to 20 values (0-19)
def quantise_pose(vector):
    return [int(round(value * 3)) for value in vector]

def process_img(row, img_parent_dir, override = False):
    img_data = None
    if override or float(row['IMAGE_DELTA_MS']) > 0:
        img_path = row['RELATIVE_IMAGE_PATH'][2:]
        full_img_path = img_parent_dir / f"{img_path}.png"
        img = cv.imread(str(full_img_path))
        img = cv.resize(img, (320, 180))
        img = np.rot90(img)
        img = np.ascontiguousarray(img)
        bounding_box = detector.detect(img)[1]
        if bounding_box is not None:
            img_data = (
                {
                    'IMG_PATH': img_path,
                    'FACE_DETECTED': True,
                    **{key: bounding_box[0][idx] for idx, key in enumerate(YUNET_FIELDS)}
                }, full_img_path
            )
        else:
            img_data = (
                {
                    'IMG_PATH': img_path,
                    'FACE_DETECTED': False,
                    **{key: None for key in YUNET_FIELDS}
                }, full_img_path
            )
    return img_data

def process_imu(row, override = False):
    imu_data = None
    if override or float(row['IMU_DELTA_MS']) > 0:
        quantised_acc = quantise_acceleration(np.array([float(row['LINEAR_X']), float(row['LINEAR_Y']), float(row['LINEAR_Z'])]))
        quantised_rot = quantise_acceleration(np.array([float(row['ROTATION_X']), float(row['ROTATION_Y']), float(row['ROTATION_Z'])]))
        imu_data = {
            'LINEAR_X': quantised_acc[0],
            'LINEAR_Y': quantised_acc[1],
            'LINEAR_X': quantised_acc[2],
            'ROTATION_X': quantised_rot[0],
            'ROTATION_Y': quantised_rot[1],
            'ROTATION_Z': quantised_rot[2],
        }
    return imu_data

def process_mocap(row, override = False):
    mocap_data = None
    if override or float(row['MOCAP_DELTA_MS']) > 0:
        phone_mocap_loc = quantise_position(np.array([float(row['PHONE_X']), float(row['PHONE_Y']), float(row['PHONE_Z'])]))
        phone_mocap_pose = quantise_pose(np.array([float(row['PHONE_ROLL']), float(row['PHONE_PITCH']), float(row['PHONE_YAW'])]))
        head_mocap_loc = quantise_position(np.array([float(row['HEAD_X']), float(row['HEAD_Y']), float(row['HEAD_Z'])]))
        head_mocap_pose = quantise_pose(np.array([float(row['HEAD_ROLL']), float(row['HEAD_PITCH']), float(row['HEAD_YAW'])]))
        mocap_data = {
            'PHONE_X': phone_mocap_loc[0],
            'PHONE_Y': phone_mocap_loc[1],
            'PHONE_Z': phone_mocap_loc[2],
            'PHONE_ROLL': phone_mocap_pose[0],
            'PHONE_PITCH': phone_mocap_pose[1],
            'PHONE_YAW': phone_mocap_pose[2],
            'HEAD_X': head_mocap_loc[0],
            'HEAD_Y': head_mocap_loc[1],
            'HEAD_Z': head_mocap_loc[2],
            'HEAD_ROLL': head_mocap_pose[0],
            'HEAD_PITCH': head_mocap_pose[1],
            'HEAD_YAW': head_mocap_pose[2],
        }
    return mocap_data

def process_row(row, file_parents, override = False):
    imu_data = process_imu(row, override)
    mocap_data = process_mocap(row, override)
    image = process_img(row, file_parents, override)
    return image, imu_data, mocap_data

pose_training_path = NN_PATH / 'PoseTraining.csv' # one file with all the 
motion_training_path = NN_PATH / 'MotionTraining.csv' # one file with all the 
with open(pose_training_path, 'w') as pose_training_file, open(motion_training_path, 'w') as motion_training_file:
    pose_training_csv = csv.DictWriter(pose_training_file, OUTPUT_CSV_FIELD_NAMES, extrasaction='ignore')
    pose_training_csv.writeheader()
    motion_training_csv = csv.DictWriter(motion_training_file, MOTION_OUTPUT_CSV_FIELD_NAMES, extrasaction='ignore')
    motion_training_csv.writeheader()

    for participant, take, gesture, direction, file in file_path_generator(DATA_PATH):
        print(f"Processing: {participant}-{take}-{gesture}-{direction}")
        file_parents = file.parents[0]
        with open(file, 'r') as csv_file:
            reader = csv.DictReader(csv_file)
            aggregated_data = {
                key: [] for key in [0, 5, 10, 15, 20] # 0 is raw, aggregate on each new image
            }
            try:
                first_row = reader.__next__()
                img, imu, mocap = process_row(first_row, file_parents, override = True)
                pose_data = {
                    **img[0],
                    **imu,
                    **mocap
                }
                pose_training_csv.writerow(pose_data)
                for key, aggregate in aggregated_data.items():
                    agg_file = f"{'RAW' if key == 0 else str(key)}_{img[1].stem}_{gesture}_{direction}_{participant}_{take}.csv"  # take the image name as the first image in sequence for given aggregate
                    pose_file = open(NN_POSE_PATH / agg_file, 'w')
                    pose_csv = csv.DictWriter(pose_file, AGG_OUTPUT_CSV_FIELD_NAMES, extrasaction='ignore') # update
                    pose_csv.writeheader()
                    motion_file = open(NN_MOTION_PATH / agg_file, 'w')
                    motion_csv = csv.DictWriter(motion_file, AGG_MOTION_OUTPUT_CSV_FIELD_NAMES, extrasaction='ignore') # update
                    motion_csv.writeheader()

                    aggregate.append((
                        [
                            (pose_file, pose_csv), 
                            (motion_file, motion_csv), 
                        ], {
                            'DELTA': 0,
                            'IMG': img[0],      # The previous image for this delta
                            'IMU': [imu],       # want to average acceleration between images.
                            'MOCAP': mocap,     # only need latest ground-truth
                            'OLD_MOCAP': mocap  # Need old to calc diff, not used for pose data
                        }
                    ))

                for row in reader:
                    delta = float(row['MS_FROM_START'])
                    img, imu, mocap = process_row(row, file_parents)

                    if imu is not None:
                        for aggregate in aggregated_data.values():
                            for window in aggregate:
                                window[1]['IMU'].append(imu)
                    
                    if mocap is not None:
                        for aggregate in aggregated_data.values():
                            for window in aggregate:
                                window[1]['MOCAP'] = mocap

                    if img is not None:
                        # check each aggregate to see if current image exceeds delta (e.g. for 5fps, whether 200ms have passed since last image, etc)
                        # For each aggregate exceeded:
                        #   calculate deltas / averages
                        #   write to appropriate file
                        for fps, aggregate in aggregated_data.items():
                            # use fps to determine if need to write to file
                            for files, agg in aggregate:
                                # check if delta diff exceeds scaled sample rate, and that we have at least some more data since last img
                                delta_diff = delta - agg['DELTA']
                                # print(f"IMU count: {len(agg['IMU'])}")
                                if (fps == 0 or delta_diff > 1000.0/fps) and len(agg['IMU']) > 1:
                                    # print(f"New data for current sample_rate: {fps == 0} {'N/A' if fps == 0 else 1000.0/fps}, {delta_diff}")
                                    imu_avg = {
                                        'LINEAR_X': 0, 'LINEAR_Y': 0, 'LINEAR_X': 0, 
                                        'ROTATION_X': 0, 'ROTATION_Y': 0, 'ROTATION_Z': 0
                                    }
                                    for imu_data in agg['IMU']:
                                        for key, acc in imu_data.items():
                                            imu_avg[key] = imu_avg[key] + acc
                                    
                                    for key in imu_avg.keys():
                                        imu_avg[key] = imu_avg[key] / len(agg['IMU'])

                                    mocap_encoding = {
                                        'PHONE_X': '', 'PHONE_Y': '', 'PHONE_Z': '', 
                                        'PHONE_ROLL': '', 'PHONE_PITCH': '', 'PHONE_YAW': '', 
                                        'HEAD_X': '', 'HEAD_Y': '', 'HEAD_Z': '', 
                                        'HEAD_ROLL': '', 'HEAD_PITCH': '', 'HEAD_YAW': ''
                                    }

                                    for key in mocap_encoding.keys():
                                        mocap_encoding[key] = encode_motion(agg['MOCAP'][key], agg['OLD_MOCAP'][key])

                                    motion_data = {
                                        'DELTA': delta_diff,
                                        **{f"{key}_1": value for key, value in agg['IMG'].items()},
                                        **{f"{key}_2": value for key, value in img[0].items()},
                                        **imu_avg,
                                        **mocap_encoding
                                    }
                                    # for key, val in agg['IMG'].items():
                                    #     motion_data[f"{key}_1"] = val
                                    # for key, val in img[0].items():
                                    #     motion_data[f"{key}_1"] = val
                                    # print(img[0], imu, mocap)
                                    pose_data = {
                                        'DELTA': delta_diff,
                                        **img[0],
                                        **imu_avg,
                                        **agg['MOCAP']
                                    }
                                    agg[1] = {
                                        'DELTA': delta,
                                        'IMG': img[0],      # The previous image for this delta
                                        'IMU': [agg['IMU'][-1]],       # want to average acceleration between images.
                                        'MOCAP': agg['MOCAP'],     # only need latest ground-truth
                                        'OLD_MOCAP': agg['MOCAP']  # Need old to calc diff, not used for pose data
                                    }

                                    pose_training_csv.writerow(pose_data)
                                    pose_training_file.flush()
                                    files[0][1].writerow(pose_data)
                                    motion_training_csv.writerow(motion_data)
                                    motion_training_file.flush()
                                    files[1][1].writerow(motion_data)

                            delta_exceeded = fps == 0 or delta > 1000.0/fps
                            # if delta not exceeded, then we want to add as another aggregate starting with this img.
                            if not delta_exceeded:
                                agg_file = f"{'RAW' if fps == 0 else str(fps)}_{img[1].stem}_{gesture}_{direction}_{participant}_{take}.csv"  # take the image name as the first image in sequence for given aggregate
                                print(f"New file: {str(agg_file)}")
                                pose_file = open(NN_POSE_PATH / agg_file, 'w')
                                pose_csv = csv.DictWriter(pose_file, AGG_OUTPUT_CSV_FIELD_NAMES, extrasaction='ignore') # update
                                pose_csv.writeheader()
                                motion_file = open(NN_MOTION_PATH / agg_file, 'w')
                                motion_csv = csv.DictWriter(motion_file, AGG_MOTION_OUTPUT_CSV_FIELD_NAMES, extrasaction='ignore') # update
                                motion_csv.writeheader()
                                pose_data = {
                                    'DELTA': 0,
                                    **img[0],
                                    **aggregate[-1][1]['IMU'][-1],
                                    **aggregate[-1][1]['MOCAP']
                                }
                                aggregate.append((
                                    [
                                        (pose_file, pose_csv), 
                                        (motion_file, motion_csv), 
                                    ], {
                                        'DELTA': 0,
                                        'IMG': img[0],      # The previous image for this delta
                                        'IMU': [aggregate[-1][1]['IMU'][-1]],       # want to average acceleration between images.
                                        'MOCAP': aggregate[-1][1]['MOCAP'],     # only need latest ground-truth
                                        'OLD_MOCAP': aggregate[-1][1]['OLD_MOCAP']  # Need old to calc diff, not used for pose data
                                    }
                                ))
                                pose_training_csv.writerow(pose_data)
                                pose_csv.writerow(pose_data)
                            # print(aggregated_data)
                for aggregate in aggregated_data.values():
                    for files, _ in aggregate:
                        for file in files:
                            file[0].flush()
                            file[0].close()
            except Exception as e:
                print(f"Uh-oh: {e}", e.with_traceback)


Processing: 0-0-CIRCULAR_HEAD-ANTI_CLOCKWISE
New file: 5_2022-08-22T09-32-22-615_CIRCULAR_HEAD_ANTI_CLOCKWISE_0_0.csv
New file: 10_2022-08-22T09-32-22-615_CIRCULAR_HEAD_ANTI_CLOCKWISE_0_0.csv
New file: 15_2022-08-22T09-32-22-615_CIRCULAR_HEAD_ANTI_CLOCKWISE_0_0.csv
New file: 20_2022-08-22T09-32-22-615_CIRCULAR_HEAD_ANTI_CLOCKWISE_0_0.csv
New file: 5_2022-08-22T09-32-22-627_CIRCULAR_HEAD_ANTI_CLOCKWISE_0_0.csv
New file: 10_2022-08-22T09-32-22-627_CIRCULAR_HEAD_ANTI_CLOCKWISE_0_0.csv
New file: 15_2022-08-22T09-32-22-627_CIRCULAR_HEAD_ANTI_CLOCKWISE_0_0.csv
New file: 20_2022-08-22T09-32-22-627_CIRCULAR_HEAD_ANTI_CLOCKWISE_0_0.csv
New file: 5_2022-08-22T09-32-22-634_CIRCULAR_HEAD_ANTI_CLOCKWISE_0_0.csv
New file: 10_2022-08-22T09-32-22-634_CIRCULAR_HEAD_ANTI_CLOCKWISE_0_0.csv
New file: 15_2022-08-22T09-32-22-634_CIRCULAR_HEAD_ANTI_CLOCKWISE_0_0.csv
New file: 20_2022-08-22T09-32-22-634_CIRCULAR_HEAD_ANTI_CLOCKWISE_0_0.csv
New file: 5_2022-08-22T09-32-22-643_CIRCULAR_HEAD_ANTI_CLOCKWISE_0_0.c